# Lesson3 系列データで分類・予測させてみよう（RNN, LSTM）

## 目次

- Section1 解説
    - 1.1 RNNとは
    - 1.2 RNNの実装
    - 1.3 BPTTとRNNの注意点
    - Checkクイズ
- Section2 実装①
    - 2.0 データの用意
    - 2.1 モデル構築
    - 2.2 モデルの学習
    - 2.3 モデルによる予測
    - 2.4 モデルの可視化
- Section3 精度向上Tips
    - 3.1 Clippingによる勾配爆発への対処
    - 3.2 ショートカットとゲートによる勾配消失への対処
    - 3.3 LSTM
    - 3.4 GRU
    - Checkクイズ
- Section4 実装②
    - 4.0 データの用意
    - 4.1 モデル構築
    - 4.2 モデルの学習
    - 4.3 モデルによる予測
    - 4.4 モデルの可視化
- Section5 ケーススタディ
    - 5.1 時系列データの異常検知
    - 5.2 画像との組み合わせ
- Homework

## Section1 解説

### 1.1 RNNとは

**再帰型ニューラルネットワーク (Recurrent Neural Network; RNN)** は、系列データを対象としたモデルです。

CNNでは平面的な位置関係を踏まえた学習が行われていましたが、RNNでは系列データの持つ（時間的な）前後関係を踏まえた学習を目的としています。

まずRNNのモデルの模式図を見てみましょう。

![RNN](figures/rnn_unfold.png "RNN")

出典：Y. LeCun, Y. Bengio, and G.  Hinton, “Deep learning,” Nature, vol. 521, no. 7553, pp. 436–444, 2015. ( https://www.nature.com/articles/nature14539 )

この図の左図がRNNを表すときによく用いられる図です。

入力$x$に対して変換$U$を適用した後、**繰り返し$W$を適用し**、最後に変換$V$を施したうえで出力しています。

この繰り返し（再帰）にあたる部分(図中のループ)を展開して分かりやすくしたものが右図です。

注意してほしいのは、**RNNの入力データ$x$は系列データ**であるということです。

つまり、例えば文章を処理する際には、文章を単語（あるいは文字）の系列であると捉え、各文章1文1文を入力データとします。

したがって、データセット自体はそうした複数の文章の集まりとなります。文章の感情分類の場合には以下のようになるというわけです。

<img src="figures/sentiment.png" width=80%>

文章に対する判別や予測においては、「ある1文がデータセット、各構成単語が入力データ」とはならないということです。

---

$x$は系列データなので、それ自体$x_1, x_2, \ldots, x_T$という前後関係を持つ要素から構成されます。（先の例で$x_t$が単語（または文字）に対応）

RNNではこの各要素$x_t$に対して$U,W,V$による変換を繰り返し行っていくわけです。

ここで最も重要な点は、**各$x_t$は独立に処理されるわけではない**ということです。

$x_t$への変換$W$の適用に当たっては、$t$以前の情報を要約したデータ（図中$s_t$）を考慮しています。（横矢印が対応）

これによって、時間的な構造が反映されるわけです。

また、使用されている変換$V,U,W$も各時点で同じものになっていることにも注意してください。（$V_t,U_t,W_t$ではない）

これも各時点が独立に処理されるのではないということの表れです。

---

こうした特徴より、RNNはもっぱら時系列データに対して応用されます。

実際、株価の予測モデルや自然言語処理などへの適用が行われています。

特に自然言語処理への適用例では目覚ましい成果も出ており、この後に扱うことになります。

### 1.2 RNNの実装

RNNの実装を行っていきますが、初めに1つ補足しておきます。

それは上図において、$V$として行っている出力の線形変換（＋活性化）は一般にRNNのモデルには含めないということです。

つまり、活性化関数を$f$として、$$o_t=f(Ux_t+Ws_{t-1})$$と出力するところまでがRNNレイヤーの役割です。

もちろん、このRNNレイヤーの直後にDenseレイヤーをつければ、上図と同様のモデルを作成できます。

それでは、このRNNレイヤーのKerasにおける実装を見ていきましょう。

といっても、再帰という複雑そうな見かけによらず、`keras.layers.SimpleRNN`を用いるだけで簡単に実装できます。

https://keras.io/ja/layers/recurrent/#simplernn

```python
keras.layers.SimpleRNN(units, activation='tanh', use_bias=True,
                       kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', 
                       kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
                       kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, 
                       dropout=0.0, recurrent_dropout=0.0, return_sequences=False, return_state=False, 
                       go_backwards=False, stateful=False, unroll=False)
```

引数は次の通りです。

* units：出力次元（上図$o_t$の次元）
* activation：活性化関数
* use_bias：バイアスベクトル（$Ux_t+Ws_{t-1}$に付け加えるベクトル）を使用するか
* {kernel,recurrent,bias}_initializer：各パラメータの初期化法（kernelは上図$U$, recurrentは上図$W$を指す）
* {kernel,recurrent,bias,activity}_regularizer：各パラメータの正則化（activityは出力=activationを指す）
* {kernel,recurrent,bias}_constraint：各パラメータに課す制約
* dropout：入力についてのdropoutの比率
* recurrent_dropout：再帰についてのdropoutの比率（上図横矢印に対して適用するdropout）
* return_sequences: Falseなら出力としては系列の最後の出力のみ（$o_T$のみ）を返す、Trueなら出力として完全な系列（$o_1,o_2,\ldots,o_T$）を返す
* return_state: Trueのときは出力とともに，最後の状態（$s_T$）を返す
* go_backwards: Trueのときは入力系列を後ろから処理する（出力も逆順に）
* stateful: Trueのときは、前バッチの各サンプルに対する最後の状態を、次のバッチのサンプルに対する初期状態として引き継ぐ
* unroll: （高速化のためのオプション）Trueのときは再帰が展開され高速化されるが、よりメモリに負荷がかかる（短い系列にのみ適する）

### 1.3 BPTTとRNNの注意点

RNNにおけるBackpropagation(誤差逆伝播、Lesson1§3.2.2参照)は、実は時間をさかのぼって誤差を逆伝播させることに対応しています。

つまり、RNN層の勾配（更新のために使用する誤差情報）は

**（RNN層の勾配）＝（t=Tの勾配）×・・・×（t=2の勾配）×（t=1の勾配）**

のように、RNN層内の各時点での勾配の積となるということです。

そこで、RNNでのBackpropagationを特に**BackPropagation Through Time(BPTT)**と呼んでいます。

このように、RNNでは時間の分だけ勾配の積が発生するため、出力付近の勾配が過大に、入力付近では勾配が過小になる傾向があります。

これをそれぞれ**勾配爆発**、**勾配消失**と呼びます。

勾配爆発が起きればオーバーフローなどで計算が不安定になります。

勾配消失が起きれば入り口付近の更新が進まなくなります。

SimpleRNNにはこのような課題があるということはひとまず認識しておきましょう。（対策については§3で扱います）

### Checkクイズ

* 問題1
    * RNNの説明として最も正しいものを選択しなさい
* 選択肢
  1. RNNは順伝播型である
  2. RNNは時系列データに適したモデルである
  3. RNNの学習時にはデータの空間的な配置が考慮される
  4. 時系列データに対してCNNなどのRNN以外のモデルは使用できない
  
  
* 問題2. 
    * RNNの実装面の説明として最も正しいものを選択しなさい
* 選択肢
  1. RNNは各時点毎に異なる行列による変換を行うことで柔軟な出力を実現している
  2. RNNに入力する時系列は各時点のデータがスカラー
  3. 教師データは時系列データでなくてもよい
  4. 出力の時点数はKerasでは`keras.layers.recurrent.SimpleRNN`のunitsで指定する